In [20]:
# Load in the python modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve

import sklearn.metrics as metrics
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# Create the data frames
student_info_df = pd.read_csv('data/studentInfo.csv')


In [32]:
student_info_df.head(20)

student_info_df.isnull().sum()

student_info_df.dropna(subset=['imd_band'], inplace=True)
student_info_df.reset_index(drop=True, inplace=True)
student_info_df.head(30)


y = student_info_df[['final_result']]
# convert pass to 0 and fail to 1 and withdrawn to 2 and distinction to 3
y = y.replace(['Pass', 'Fail', 'Withdrawn', 'Distinction'], [0, 1, 2, 3])

# create a new data frame called x which might be used to predict the final result

x = student_info_df.drop(['final_result', 'code_module', 'code_presentation', 'id_student'], axis=1)
#x = pd.get_dummies(x, columns=['disability', 'imd_band', 'age_band','studied_credits','region', 'highest_education', 'gender'])
# # convert the categorical data to numerical data for everything except studied_credits
le = LabelEncoder()
x['gender'] = le.fit_transform(x['gender'])
x['disability'] = le.fit_transform(x['disability'])
x['imd_band'] = le.fit_transform(x['imd_band'])
x['age_band'] = le.fit_transform(x['age_band'])
x['region'] = le.fit_transform(x['region'])
x['highest_education'] = le.fit_transform(x['highest_education'])

# use a descision tree to predict the final result
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
clf = DecisionTreeClassifier()

# fit the model
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

# print the accuracy of the model
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

predicted = cross_val_score(clf, x, y, cv=10)
print(predicted.mean())


Accuracy: 0.3529738743746526
0.35807770402092104
